In [1]:
# !wget https://huggingface.co/datasets/mesolitica/kesalahan-tatabahasa/resolve/main/train.jsonl -O tatabahasa1.jsonl
# !wget https://huggingface.co/datasets/mesolitica/kesalahan-tatabahasa/resolve/main/train2.jsonl -O tatabahasa2.jsonl

In [2]:
describe = [
    {'class': 0,
     'Description': 'PAD',
     'salah': '',
     'betul': ''},
    {'class': 1,
     'Description': 'kesambungan subwords',
     'salah': '',
     'betul': '',
     },
    {'class': 2,
     'Description': 'tiada kesalahan',
     'salah': '',
     'betul': '',
     },
    {'class': 3,
     'Description': 'kesalahan frasa nama, Perkara yang diterangkan mesti mendahului "penerang"',
     'salah': 'Cili sos',
     'betul': 'sos cili',
     },
    {'class': 4,
     'Description': 'kesalahan kata jamak',
     'salah': 'mereka-mereka',
     'betul': 'mereka',
     },
    {'class': 5,
     'Description': 'kesalahan kata penguat',
     'salah': 'sangat tinggi sekali',
     'betul': 'sangat tinggi',
     },
    {'class': 6,
     'Description': 'kata adjektif dan imbuhan "ter" tanpa penguat',
     'salah': 'Sani mendapat markah yang tertinggi sekali.',
     'betul': 'Sani mendapat markah yang tertinggi.',
     },
    {'class': 7,
     'Description': 'kesalahan kata hubung',
     'salah': 'Sally sedang membaca bila saya tiba di rumahnya.',
     'betul': 'Sally sedang membaca apabila saya tiba di rumahnya.',
     },
    {'class': 8,
     'Description': 'kesalahan kata bilangan',
     'salah': 'Beribu peniaga tidak membayar cukai pendapatan.',
     'betul': 'Beribu-ribu peniaga tidak membayar cukai pendapatan',
     },
    {'class': 9,
     'Description': 'kesalahan kata sendi',
     'salah': 'Umar telah berpindah daripada sekolah ini bulan lalu.',
     'betul': 'Umar telah berpindah dari sekolah ini bulan lalu.',
     },
    {'class': 10,
     'Description': 'kesalahan penjodoh bilangan',
     'salah': 'Setiap orang pelajar',
     'betul': 'Setiap pelajar.',
     },
    {'class': 11,
     'Description': 'kesalahan kata ganti diri',
     'salah': 'Pencuri itu telah ditangkap. Beliau dibawa ke balai polis.',
     'betul': 'Pencuri itu telah ditangkap. Dia dibawa ke balai polis.',
     },
    {'class': 12,
     'Description': 'kesalahan ayat pasif',
     'salah': 'Cerpen itu telah dikarang oleh saya.',
     'betul': 'Cerpen itu telah saya karang.',
     },
    {'class': 13,
     'Description': 'kesalahan kata tanya',
     'salah': 'Kamu berasal dari manakah ?',
     'betul': 'Kamu berasal dari mana ?',
     },
    {'class': 14,
     'Description': 'kesalahan tanda baca',
     'salah': 'Kamu berasal dari manakah .',
     'betul': 'Kamu berasal dari mana ?',
     },
    {'class': 15,
     'Description': 'kesalahan kata kerja tak transitif',
     'salah': 'Dia kata kepada saya',
     'betul': 'Dia berkata kepada saya',
     },
    {'class': 16,
     'Description': 'kesalahan kata kerja transitif',
     'salah': 'Dia suka baca buku',
     'betul': 'Dia suka membaca buku',
     },
    {'class': 17,
     'Description': 'penggunaan kata yang tidak tepat',
     'salah': 'Tembuk Besar negeri Cina dibina oleh Shih Huang Ti.',
     'betul': 'Tembok Besar negeri Cina dibina oleh Shih Huang Ti',
     },
]

kesalahan = {}
for d in describe[2:]:
    kesalahan[d['class']] = d['Description']
    
len(kesalahan)

16

In [3]:
from collections import defaultdict
from glob import glob
from tqdm import tqdm
import json
import random

import re

# from https://huggingface.co/Qwen/QwQ-32B-Preview

def fix_spacing(text):
    # Handle double quotes: strip spaces inside quotes
    quote_pattern = r'"([^"]*)"'
    def fix_quotes(match):
        content = match.group(1).strip()
        return f'"{content}"'
    text = re.sub(quote_pattern, fix_quotes, text)
    
    # Handle parentheses: strip spaces inside parentheses
    paren_pattern = r'\(([^)]*)\)'
    def fix_parens(match):
        content = match.group(1).strip()
        return f'({content})'
    text = re.sub(paren_pattern, fix_parens, text)
    
    # Remove spaces before standalone punctuation: ',', '.', etc.
    punct_pattern = r'\s+([,.!:])'
    text = re.sub(punct_pattern, r'\1', text)
    
    return text.replace(' ()', '')

In [4]:
before_after = []
for f in ['tatabahasa1.jsonl', 'tatabahasa2.jsonl']:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            
            if len(l[0]) < 5:
                continue
            if len(l[0]) != len(l[1]):
                continue
                
            before = ' '.join([l_[0] for l_ in l[1]])
            after = ' '.join([l_[0] for l_ in l[0]])
            
            before = fix_spacing(before)
            after = fix_spacing(after)
            
            if random.random() > 0.7:
                before = after
                
            before_after.append({
                'before': before,
                'after': after
            })
            
len(before_after)

497050it [00:09, 53137.84it/s]
497079it [00:09, 52601.26it/s]


943378

In [5]:
slides = []

i = 0
while i < len(before_after):
    k = random.randint(2, 20)
    slide = before_after[i: i + k]
    slides.append(slide)
    i += k
    
len(slides)

85996

In [6]:
slides[0]

[{'before': 'Dirk Jan Klaas "Klaas-Jan" Huntelaar (lahir 12 Ogos 1983) merupakan pemain bola sepak Belanda yang main di posisi penyerang.',
  'after': 'Dirk Jan Klaas "Klaas-Jan" Huntelaar (lahir 12 Ogos 1983) merupakan pemain bola sepak Belanda yang bermain di posisi penyerang.'},
 {'before': 'Beliau sekarang memainkan untuk pertubuhan Ajax ;',
  'after': 'Beliau kini bermain untuk kelab Ajax.'},
 {'before': 'Hypo-Arena (dahulu dikenali sebagai) ialah sebuah stadium guna serba terhadap Klagenfurt, Austria.',
  'after': 'Hypo-Arena (dahulu dikenali sebagai) ialah sebuah stadium serba guna di Klagenfurt, Austria.'},
 {'before': 'Ia merupakan stadium kuntum Austria Karnten.',
  'after': 'Ia merupakan stadium pasukan Austria Karnten.'},
 {'before': 'Stadium lama dikenali sebagai Wortherseestadion ; dibina pada 1960 dan mempunyai kapasiti sebanyak 10,900 ;',
  'after': 'Stadium lama dikenali sebagai Wortherseestadion, dibina pada 1960 dan mempunyai kapasiti sebanyak 10,900.'},
 {'before': 

In [7]:
data = []

prompts = [
    'betulkan kesalahan tatabahasa\n```\n{string}\n```',
    '```\n{string}\n```\nbetulkan kesalahan tatabahasa',
    '```\n{string}\n```\nfix the tatabahasa',
]

no_fix = [
    'Tiada kesalahan tatabahasa untuk teks yang diberikan.',
    '{string}\n\nTiada kesalahan tatabahasa untuk teks yang diberikan.'
]

for d in before_after:
    if d['before'] == d['after']:
        a = random.choice(no_fix).format(string = d['after'])
    else:
        a = d['after']
        
    d = {
        'prompt_input': None,
        'input': random.choice(prompts).format(string = d['before']),
        'output': a,
    }
    data.append(d)
    
len(data)

943378

In [8]:
data[0]

{'prompt_input': None,
 'input': '```\nDirk Jan Klaas "Klaas-Jan" Huntelaar (lahir 12 Ogos 1983) merupakan pemain bola sepak Belanda yang main di posisi penyerang.\n```\nfix the tatabahasa',
 'output': 'Dirk Jan Klaas "Klaas-Jan" Huntelaar (lahir 12 Ogos 1983) merupakan pemain bola sepak Belanda yang bermain di posisi penyerang.'}

In [9]:
import pandas as pd

df = pd.DataFrame(data)
df.to_parquet('prepared-kesalahan-tatabahasa-sentence-fix.parquet')

In [10]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-kesalahan-tatabahasa-sentence-fix.parquet",
    path_in_repo='data/kesalahan_tatabahasa_sentence_fix-00000-of-00001.parquet',
    repo_id="mesolitica/kesalahan-tatabahasa-instructions",
    repo_type="dataset",
)

prepared-kesalahan-tatabahasa-sentence-fix.parquet:   0%|          | 0.00/134M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/kesalahan-tatabahasa-instructions/commit/3bfbaa64fc8db5daafa89dcae8ad1cfbe832a295', commit_message='Upload data/kesalahan_tatabahasa_sentence_fix-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='3bfbaa64fc8db5daafa89dcae8ad1cfbe832a295', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
data = []

for s in slides:
    before = [s_['before'] for s_ in s]
    after = [s_['after'] for s_ in s]
    before = ' '.join(before)
    after = ' '.join(after)
    
    if before == after:
        a = random.choice(no_fix).format(string = after)
    else:
        a = after
    
    d = {
        'prompt_input': None,
        'input': random.choice(prompts).format(string = before),
        'output': a,
    }
    data.append(d)

In [12]:
df = pd.DataFrame(data)
df.to_parquet('prepared-kesalahan-tatabahasa-slides-sentence-fix.parquet')

In [13]:
api.upload_file(
    path_or_fileobj="prepared-kesalahan-tatabahasa-slides-sentence-fix.parquet",
    path_in_repo='data/kesalahan_tatabahasa_slides_sentence_fix-00000-of-00001.parquet',
    repo_id="mesolitica/kesalahan-tatabahasa-instructions",
    repo_type="dataset",
)

prepared-kesalahan-tatabahasa-slides-sentence-fix.parquet:   0%|          | 0.00/134M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/kesalahan-tatabahasa-instructions/commit/47d75d3f033fd92b03eb03fe60afb7ed2a706552', commit_message='Upload data/kesalahan_tatabahasa_slides_sentence_fix-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='47d75d3f033fd92b03eb03fe60afb7ed2a706552', pr_url=None, pr_revision=None, pr_num=None)